<a href="https://colab.research.google.com/github/phanikiran99/kaggle/blob/master/Covid19India_Predicting_Future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import requests

#plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objects as go

import seaborn as sns
sns.set()

import matplotlib.pyplot as plt
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [0]:
api = 'https://api.covid19india.org/data.json'
resp = requests.get(api)

In [0]:
caseTimeSeries = pd.DataFrame(resp.json()['cases_time_series'])
stateWise = pd.DataFrame(resp.json()['statewise'])

In [0]:
caseTimeSeries.to_excel('caseTimeSeries.xlsx')

In [0]:
caseTimeSeries['meanTotal'] = caseTimeSeries['totalconfirmed'].rolling(window=5).mean()
caseTimeSeries['meanRecovered'] = caseTimeSeries['totalrecovered'].rolling(window=5).mean()

In [6]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=caseTimeSeries['date'], y=caseTimeSeries['meanTotal'],name='MAVG Total Cases'))
fig.add_trace(go.Scatter(x=caseTimeSeries['date'], y=caseTimeSeries['totalconfirmed'], name='Total'))
fig.add_trace(go.Scatter(x=caseTimeSeries['date'], y=caseTimeSeries['totalrecovered'], name='Recovered'))
fig.add_trace(go.Scatter(x=caseTimeSeries['date'], y=caseTimeSeries['meanRecovered'], name='MAVG Recovered'))

fig.show()

In [0]:
from dateutil.parser import parse
import datetime as dt

In [0]:
caseTimeSeries['day'] = caseTimeSeries['date'].apply(lambda x: parse(x).date().day)
caseTimeSeries['month'] = caseTimeSeries['date'].apply(lambda x: parse(x).date().month)                                                

In [9]:
caseTimeSeries.head()

,dailyconfirmed,dailydeceased,dailyrecovered,date,totalconfirmed,totaldeceased,totalrecovered,meanTotal,meanRecovered,day,month
0,1,0,0,30 January,1,0,0,NaN,NaN,30,1
1,0,0,0,31 January,1,0,0,NaN,NaN,31,1
2,0,0,0,01 February,1,0,0,NaN,NaN,1,2
3,1,0,0,02 February,2,0,0,NaN,NaN,2,2
4,1,0,0,03 February,3,0,0,1.6,0.0,3,2


In [0]:
y  = caseTimeSeries['dailyconfirmed']
X = pd.DataFrame(caseTimeSeries[['day','month']])

In [0]:
from sklearn.linear_model import LinearRegression

In [14]:
log_model = LinearRegression()
log_model.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
import datetime as dt
base = dt.datetime.today()
date_list = [base - dt.timedelta(days=x) for x in range(90)]
date_list

date_dict = {'day':[x.day for x in date_list],'month':[x.month for x in date_list]}

In [0]:
X_Future = pd.DataFrame(date_dict)

In [0]:
y_fut = log_model.predict(X_Future)

In [16]:
X.index.max()+X_Future.index

RangeIndex(start=94, stop=184, step=1)

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=X.index, y=y, name='Actuals Cases till Date'))
fig.add_trace(go.Scatter(x=X.index.max()+X_Future.index, y=y_fut, name='Predictions'))
fig.update_layout(title='Forecast of Predictions Covid19 India')